In [1]:
# Import neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import neurokit2 as nk  # For ECG and HRV analysis

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, auc

import pickle

# Set plotting style
plt.style.use('dark_background')

In [2]:
# Data Loading

# Load ECG data for 1 subject, day 4
file_id = '1URua1BRmcTgkwU1nGPWRkrxhabWVotxH'
url = f'https://drive.google.com/uc?id={file_id}&export=download'
df_tochunk = pd.read_feather(url)  # ECG data

# Load glucose data for the same subject, all days
file_id = '1qGfSIb9EEJ4ZxlWnBcsILgh9LbHAiMld'
url = f'https://drive.google.com/uc?id={file_id}&export=download'
gl_d1 = pd.read_feather(url)  # Glucose data

In [3]:
df_tochunk.tail()

,EcgWaveform
datetime,
2014-10-04 13:48:01.403,1956
2014-10-04 13:48:01.407,1956
2014-10-04 13:48:01.411,1957
2014-10-04 13:48:01.415,1957
2014-10-04 13:48:01.419,1957


In [4]:
gl_d1.tail()

,glucose,type
datetime,,
2014-10-06 16:34:02,12.3,cgm
2014-10-06 16:39:02,12.5,cgm
2014-10-06 16:44:02,12.5,cgm
2014-10-06 16:49:02,12.3,cgm
2014-10-06 16:54:02,12.1,cgm


In [5]:
gl_d1.head()

,glucose,type
datetime,,
2014-10-01 19:14:00,10.3,cgm
2014-10-01 19:19:00,9.9,cgm
2014-10-01 19:23:00,9.4,manual
2014-10-01 19:24:00,9.8,cgm
2014-10-01 19:29:00,9.6,cgm


In [6]:
# Get the time range of the ECG data
ecg_start_time = df_tochunk.index.min()
ecg_end_time = df_tochunk.index.max()

# Filter glucose data to match the ECG time range
gl_d1_filtered = gl_d1[(gl_d1.index >= ecg_start_time) & (gl_d1.index <= ecg_end_time)]

In [7]:
gl_d1_filtered

,glucose,type
datetime,,
2014-10-04 06:39:01,6.9,cgm
2014-10-04 06:44:01,7.3,cgm
2014-10-04 06:49:01,7.8,cgm
2014-10-04 06:54:01,8.3,cgm
2014-10-04 06:59:01,8.8,cgm
...,...,...
2014-10-04 13:24:01,5.9,cgm
2014-10-04 13:29:01,6.3,cgm
2014-10-04 13:34:01,6.7,cgm


In [8]:
def chunkify_with_time(df: pd.DataFrame, chunk_size: int, step_size: int) -> list[tuple[pd.DataFrame, pd.Timestamp, pd.Timestamp]]:
    """
    Splits sensor data into overlapping chunks and returns chunks with their time ranges.
    """
    chunks = []
    for start in range(0, df.shape[0], step_size):
        end = start + chunk_size
        if end < df.shape[0]:
            chunk = df.iloc[start:end]
        else:
            chunk = df.iloc[start:]
        chunks.append((chunk, chunk.index[0], chunk.index[-1]))
    return chunks

# Define chunk size and step size
CHUNK_SIZE = 60 * 60 * 250  # 60 minutes in samples
STEP_SIZE = 60 * 250  # 60 seconds in samples

# Chunk the ECG data with time ranges
chunks_with_time = chunkify_with_time(df_tochunk, CHUNK_SIZE, STEP_SIZE)

In [9]:
chunks_with_time

[(                         EcgWaveform
  datetime                            
  2014-10-04 06:34:57.423          297
  2014-10-04 06:34:57.427          297
  2014-10-04 06:34:57.431          297
  2014-10-04 06:34:57.435          297
  2014-10-04 06:34:57.439          297
  ...                              ...
  2014-10-04 07:34:57.403         2052
  2014-10-04 07:34:57.407         2051
  2014-10-04 07:34:57.411         2050
  2014-10-04 07:34:57.415         2049
  2014-10-04 07:34:57.419         2049
  
  [900000 rows x 1 columns],
  Timestamp('2014-10-04 06:34:57.423000'),
  Timestamp('2014-10-04 07:34:57.419000')),
 (                         EcgWaveform
  datetime                            
  2014-10-04 06:35:57.423         2099
  2014-10-04 06:35:57.427         2099
  2014-10-04 06:35:57.431         2099
  2014-10-04 06:35:57.435         2099
  2014-10-04 06:35:57.439         2098
  ...                              ...
  2014-10-04 07:35:57.403         2040
  2014-10-04 07:35:57.4

In [10]:
def generate_target_labels_aligned(chunks_with_time, glucose_df, threshold=3.9, forecast_window=pd.Timedelta(minutes=30)):
    """
    Generates binary target labels for each chunk based on aligned glucose data.
    """
    target_labels = []
    for chunk, start_time, end_time in chunks_with_time:
        # Get glucose data within the chunk's time range
        glucose_in_chunk = glucose_df[(glucose_df.index >= start_time) & (glucose_df.index <= end_time)]
        
        # Check if hypoglycemia occurs within the forecast window after the chunk
        label = 0
        for onset_time in glucose_df[glucose_df['glucose'] <= threshold].index:
            if end_time < onset_time <= end_time + forecast_window:
                label = 1
                break
        target_labels.append(label)
    return target_labels

# Generate labels for aligned chunks
target_labels = generate_target_labels_aligned(chunks_with_time, gl_d1_filtered)

In [11]:
target_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [12]:
#Show me where target labels == 1
indices_with_label_1 = [i for i, label in enumerate(target_labels) if label == 1]
print(f"Indices where target_labels == 1: {indices_with_label_1}")
print(f"Total count: {len(indices_with_label_1)}")
print(f"Total labels: {len(target_labels)}")
print(f"Label distribution: {sum(target_labels)} ones, {len(target_labels) - sum(target_labels)} zeros")

Indices where target_labels == 1: [71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324]
Total count: 165
Total labels: 434
Label distribution: 165 ones, 269 zeros


In [ ]:
from sklearn.model_selection import train_test_split as sklearn_train_test_split

def train_test_split_chunks_stratified(chunks: list[pd.DataFrame], target_labels: list[int], test_size: float = 0.2, random_state: int = 42) -> tuple:
    """
    Splits chunks and labels into train and test sets using stratified sampling.
    This ensures both sets have similar proportions of positive/negative labels.
    """
    indices = list(range(len(chunks)))
    
    # Stratified split to maintain class distribution
    train_indices, test_indices = sklearn_train_test_split(
        indices, 
        test_size=test_size, 
        stratify=target_labels,
        random_state=random_state
    )
    
    X_train = [chunks[i] for i in train_indices]
    X_test = [chunks[i] for i in test_indices]
    y_train = [target_labels[i] for i in train_indices]
    y_test = [target_labels[i] for i in test_indices]
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Extract chunks without time ranges for training/testing
aligned_chunks = [chunk for chunk, _, _ in chunks_with_time]

# Split the data into train and test sets WITH STRATIFICATION
X_train, X_test, y_train, y_test = train_test_split_chunks_stratified(
    aligned_chunks, 
    target_labels, 
    test_size=0.2,
    random_state=42
)

# Verify the split has positive samples in both sets
print(f"Train set: {len(X_train)} samples, {sum(y_train)} positive ({sum(y_train)/len(y_train)*100:.1f}%)")
print(f"Test set: {len(X_test)} samples, {sum(y_test)} positive ({sum(y_test)/len(y_test)*100:.1f}%)")

### Feature Engineering

In [15]:
# Time-domain Feature Extraction
### Replace with extract_ecg_features() and extract_hrv_features() for more relevant features

def extract_features(chunks: list[pd.DataFrame]) -> pd.DataFrame:
    """
    Extracts statistical features (mean, std, min, max, etc.) from each chunk.
    """
    feature_rows = []
    for chunk in chunks:
        feats = pd.concat([
            chunk.mean().add_suffix("_mean"),
            chunk.std().add_suffix("_std"),
            chunk.min().add_suffix("_min"),
            chunk.max().add_suffix("_max"),
            chunk.quantile(0.25).add_suffix("_q25"),
            chunk.median().add_suffix("_median"),
            chunk.quantile(0.75).add_suffix("_q75"),
            chunk.skew().add_suffix("_skew"),
            chunk.kurtosis().add_suffix("_kurtosis")
        ])
        feature_rows.append(feats)
    return pd.DataFrame(feature_rows)

# Extract features for train and test sets
X_train_features = extract_features(X_train)
X_test_features = extract_features(X_test)

In [ ]:
# Morphological Feature Extraction
def extract_ecg_features(chunks: list[pd.DataFrame], verbose: bool = True) -> pd.DataFrame:
    """
    Extracts ECG waveform features (time-domain and morphological features) from each chunk.
    """
    feature_rows = []
    for i, chunk in enumerate(chunks):
        if verbose:
            print(f"Extracting ECG features: chunk {i+1}/{len(chunks)}")
        
        try:
            # Preprocess the ECG signal
            ecg_signal = chunk['EcgWaveform']  
            ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=250)

            # Extract R-peaks
            rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=250)[1]
            rpeak_indices = rpeaks['ECG_R_Peaks']

            # Calculate RR intervals (in milliseconds)
            rr_intervals = np.diff(rpeak_indices) / 250 * 1000

            # Calculate heart rate (bpm)
            heart_rates = 60000 / rr_intervals if len(rr_intervals) > 0 else np.array([np.nan])

            # Extract time-domain statistics
            stats = pd.Series({
                # Basic ECG statistics
                "mean_ecg": ecg_signal.mean(),
                "std_ecg": ecg_signal.std(),
                "min_ecg": ecg_signal.min(),
                "max_ecg": ecg_signal.max(),
                # R-peak features
                "num_rpeaks": len(rpeak_indices),
                "mean_rr_interval": np.mean(rr_intervals) if len(rr_intervals) > 0 else np.nan,
                "std_rr_interval": np.std(rr_intervals) if len(rr_intervals) > 0 else np.nan,
                # Heart rate features
                "mean_heart_rate": np.mean(heart_rates),
                "std_heart_rate": np.std(heart_rates),
                "min_heart_rate": np.min(heart_rates),
                "max_heart_rate": np.max(heart_rates),
            })
        except Exception as e:
            if verbose:
                print(f"Error processing chunk {i}: {e}")
            stats = pd.Series({
                "mean_ecg": np.nan,
                "std_ecg": np.nan,
                "min_ecg": np.nan,
                "max_ecg": np.nan,
                "num_rpeaks": np.nan,
                "mean_rr_interval": np.nan,
                "std_rr_interval": np.nan,
                "mean_heart_rate": np.nan,
                "std_heart_rate": np.nan,
                "min_heart_rate": np.nan,
                "max_heart_rate": np.nan,
            })

        # Combine features
        feature_rows.append(stats)

    return pd.DataFrame(feature_rows)

In [ ]:
def extract_hrv_features(chunks: list[pd.DataFrame], verbose: bool = True) -> pd.DataFrame:
    """
    Extracts HRV features (time-domain, frequency-domain, and nonlinear features) from each chunk.
    """
    feature_rows = []
    for i, chunk in enumerate(chunks):
        if verbose:
            print(f"Extracting HRV features: chunk {i+1}/{len(chunks)}")
        
        try:
            # Preprocess the ECG signal
            ecg_signal = chunk['EcgWaveform'] 
            ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=250)

            # Extract R-peaks
            rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=250)[1]

            # Extract HRV features
            hrv_features = nk.hrv(rpeaks, sampling_rate=250, show=False)
            hrv_features = hrv_features.iloc[0]  # Convert single-row DataFrame to Series
        except Exception as e:
            if verbose:
                print(f"Error processing chunk {i}: {e}")
            hrv_features = pd.Series(dtype=float)

        # Append HRV features
        feature_rows.append(hrv_features)

    return pd.DataFrame(feature_rows).reset_index(drop=True)

In [21]:
extract_ecg_features([X_train[0]])

,mean_ecg,std_ecg,min_ecg,max_ecg
0,1994.801601,192.068531,6.0,4088.0


In [24]:
extract_hrv_features([X_train[0]])

/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/neurokit2/hrv/hrv_time.py:242: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[    HRV_MeanNN     HRV_SDNN   HRV_SDANN1   HRV_SDNNI1  HRV_SDANN2  \
 0  3661.993884  6238.792841  6755.390009  6371.251302  4162.49674   
 
     HRV_SDNNI2   HRV_SDANN5   HRV_SDNNI5    HRV_RMSSD     HRV_SDSD  ...  \
 0  6722.779989  2836.630257  6699.451518  7807.697092  7811.683661  ...   
 
    HRV_SampEn  HRV_ShanEn  HRV_FuzzyEn  HRV_MSEn  HRV_CMSEn  HRV_RCMSEn  \
 0    0.535027    9.090536     0.660706  0.630183   0.780071    0.655891   
 
      HRV_CD   HRV_HFD   HRV_KFD   HRV_LZC  
 0  0.510656  1.955492  2.293268  0.800317  
 
 [1 rows x 91 columns]]

In [25]:

# Combine ECG and HRV features
def extract_combined_features(chunks: list[pd.DataFrame]) -> pd.DataFrame:
    """
    Combines ECG waveform features and HRV features into a single feature set.
    """
    ecg_features = extract_ecg_features(chunks)
    hrv_features = extract_hrv_features(chunks)

    # Combine both feature sets
    combined_features = pd.concat([ecg_features, hrv_features], axis=1)
    return combined_features



In [33]:
from multiprocessing import Pool

def process_chunk(chunk):
    """
    Process a single chunk to extract ECG and HRV features.
    """
    ecg_signal = chunk['EcgWaveform']
    ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=250)
    
    # Extract R-peaks (only once)
    rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=250)[1]
    
    # Extract ECG time-domain statistics
    ecg_stats = pd.Series({
        "mean_ecg": ecg_signal.mean(),
        "std_ecg": ecg_signal.std(),
        "min_ecg": ecg_signal.min(),
        "max_ecg": ecg_signal.max(),
    })
    
    # Extract HRV features
    try:
        hrv_features = nk.hrv(rpeaks, sampling_rate=250, show=False)
        hrv_features = hrv_features.iloc[0]  # Convert single-row DataFrame to Series
    except Exception as e:
        # Return NaN values if HRV extraction fails
        hrv_features = pd.Series(dtype=float)
    
    # Combine features
    combined = pd.concat([ecg_stats, hrv_features])
    return combined


def extract_combined_features_sequential(chunks: list[pd.DataFrame]) -> pd.DataFrame:
    """
    Extract combined ECG and HRV features sequentially (for debugging).
    """
    results = []
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}")
        try:
            result = process_chunk(chunk)
            results.append(result)
        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
            results.append(pd.Series(dtype=float))
    return pd.DataFrame(results).reset_index(drop=True)


In [34]:
## Use sequential version for debugging
X_train_combined_features = extract_combined_features_sequential(X_train)
X_test_combined_features = extract_combined_features_sequential(X_test)

Processing chunk 1/347


/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/neurokit2/hrv/hrv_time.py:242: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing chunk 2/347


/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing chunk 3/347


...

/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing chunk 329/347
Processing chunk 330/347
Processing chunk 331/347
Processing chunk 332/347
Processing chunk 333/347
Processing chunk 334/347
Processing chunk 335/347
Processing chunk 336/347
Processing chunk 337/347
Processing chunk 338/347
Processing chunk 339/347
Processing chunk 340/347
Processing chunk 341/347
Processing chunk 342/347
Processing chunk 343/347
Processing chunk 344/347
Processing chunk 345/347
Processing chunk 346/347
Processing chunk 347/347
Processing chunk 1/87
Processing chunk 2/87
Processing chunk 3/87
Processing chunk 4/87
Processing chunk 5/87
Processing chunk 6/87
Processing chunk 7/87
Processing chunk 8/87
Processing chunk 9/87
Processing chunk 10/87
Processing chunk 11/87
Processing chunk 12/87
Processing chunk 13/87
Processing chunk 14/87
Processing chunk 15/87
Processing chunk 16/87
Processing chunk 17/87
Processing chunk 18/87
Processing chunk 19/87
Processing chunk 20/87
Processing chunk 21/87
Processing chunk 22/87
Processing chunk 23/87
Proce

/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/neurokit2/hrv/hrv_nonlinear.py:536: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/neurokit2/hrv/hrv_nonlinear.py:536: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


Processing chunk 84/87
Processing chunk 85/87
Processing chunk 86/87
Processing chunk 87/87


/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/neurokit2/hrv/hrv_nonlinear.py:536: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/neurokit2/hrv/hrv_nonlinear.py:536: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/neurokit2/complexity/entropy_multiscale.py:349: RuntimeWarning: invalid value encountered in scalar divide
  mse 

### Model Training

In [37]:
# Handle NaN values - fill with column means or drop rows
print(f"NaN values in X_train: {X_train_combined_features.isna().sum().sum()}")
print(f"NaN values in X_test: {X_test_combined_features.isna().sum().sum()}")

NaN values in X_train: 546
NaN values in X_test: 143


In [38]:
X_train_combined_features = X_train_combined_features.fillna(X_train_combined_features.mean())
X_test_combined_features = X_test_combined_features.fillna(X_train_combined_features.mean())  # Use train means for test

In [39]:
# Handle NaN values - fill with column means or drop rows
print(f"NaN values in X_train: {X_train_combined_features.isna().sum().sum()}")
print(f"NaN values in X_test: {X_test_combined_features.isna().sum().sum()}")

NaN values in X_train: 0
NaN values in X_test: 0


In [40]:
#Define and train an SVM model
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf", C=10.0, gamma="scale", probability=True))
])
svm_clf.fit(X_train_combined_features, y_train)

,steps,"[('scaler', ...), ('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,C,10.0
,kernel,'rbf'
,degree,3
,gamma,'scale'


### Model evaluation

In [41]:
# Evaluate the model using Precision-Recall AUC
y_train_pred_proba = svm_clf.predict_proba(X_train_combined_features)[:, 1]
precision, recall, _ = precision_recall_curve(y_train, y_train_pred_proba)
pr_auc = auc(recall, precision)
print(f"Scoring PR-AUC on y_train: {pr_auc:.4f}")

y_test_pred_proba = svm_clf.predict_proba(X_test_combined_features)[:, 1]
precision, recall, _ = precision_recall_curve(y_test, y_test_pred_proba)
pr_auc = auc(recall, precision)
print(f"Scoring PR-AUC on y_test: {pr_auc:.4f}")

Scoring PR-AUC on y_train: 0.9999
Scoring PR-AUC on y_test: 0.5000


/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [42]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Predict on test data
y_test_pred = svm_clf.predict(X_test_combined_features)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Generate classification report
print(classification_report(y_test, y_test_pred))

Test Accuracy: 0.8161
              precision    recall  f1-score   support

           0       1.00      0.82      0.90        87
           1       0.00      0.00      0.00         0

    accuracy                           0.82        87
   macro avg       0.50      0.41      0.45        87
weighted avg       1.00      0.82      0.90        87



/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/jenniferdanielonwuchekwa/.pyenv/versions/hypopredict/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this b

In [43]:
# Save the trained SVM model
import os

# Ensure the directory exists
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)

# Save the model
with open(os.path.join(model_dir, 'jnnyvyr_svm_model02.pkl'), 'wb') as f:
    pickle.dump(svm_clf, f)

### Load saved model and Predict on New X_test

In [45]:
# Load the saved model
with open('../models/jnnyvyr_svm_model02.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Predict on test data
predictions = loaded_model.predict(X_test_combined_features)
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [46]:
X_test_combined_features

,mean_ecg,std_ecg,min_ecg,max_ecg,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,1981.419878,113.700492,157.0,3907.0,1479.587001,1650.370055,1161.321565,1596.899060,927.198065,1612.754016,...,0.553250,8.562476,0.689619,0.571946,0.853581,0.701904,0.591586,1.968272,2.607229,0.777274
1,1981.192236,113.002445,869.0,3907.0,1485.583782,1723.010518,1226.894766,1681.452422,988.818123,1714.299661,...,0.543846,8.567200,0.679319,0.566040,0.832528,0.683092,0.541569,1.974281,2.502390,0.785847
2,1981.378200,114.282679,869.0,3907.0,1491.210614,1745.967976,1232.122096,1705.538260,1027.923891,1719.069363,...,0.539086,8.563655,0.672694,0.552937,0.829155,0.679685,0.539772,1.973206,2.792441,0.801241
3,1981.316801,116.738077,869.0,3907.0,1498.964942,1772.016736,1289.285388,1734.378537,1054.001936,1768.198771,...,0.534756,8.560048,0.665566,0.518373,0.824786,0.676607,0.535661,1.971599,2.516061,0.796532
4,1981.333033,116.766462,869.0,3907.0,1527.302425,1875.571753,1622.422326,1868.490461,1260.894013,1879.631989,...,0.513174,8.568194,0.647651,0.471275,0.796524,0.652925,0.535812,1.973602,2.502445,0.781219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1975.860393,242.893072,284.0,3916.0,1910.929134,2305.140120,1329.113333,2404.702059,11012.379504,9086.588192,...,1.105448,6.778016,0.858386,0.994295,1.155660,1.063150,1.101653,1.980045,2.015486,1.100580
83,1979.118630,201.167979,284.0,3916.0,2156.190476,2618.730082,1293.820959,2703.334057,11012.379504,9086.588192,...,1.189584,6.273270,0.897344,1.010712,1.115398,1.008022,1.207608,1.967580,1.979121,0.989287
84,1983.616452,236.713606,284.0,3916.0,2327.094340,3011.321677,11761.284893,7449.824659,11012.379504,9086.588192,...,1.048780,5.614713,0.885722,1.081433,0.870406,0.910599,1.084176,1.942572,1.841432,0.972666
85,1987.850563,319.228721,284.0,3916.0,3713.411765,4681.635172,11761.284893,7449.824659,11012.379504,9086.588192,...,1.252763,4.087463,1.164410,0.677022,0.685163,0.573480,1.313071,1.949934,2.002946,1.202195


In [ ]:
X_train_features

,EcgWaveform_mean,EcgWaveform_std,EcgWaveform_min,EcgWaveform_max,EcgWaveform_q25,EcgWaveform_median,EcgWaveform_q75,EcgWaveform_skew,EcgWaveform_kurtosis
0,1994.801601,192.068531,6.0,4088.0,1941.0,1979.0,2015.0,2.027784,35.374922
1,1990.102278,147.017435,6.0,4088.0,1940.0,1979.0,2013.0,2.107751,32.375335
2,1987.327432,140.040414,6.0,4088.0,1940.0,1978.0,2011.0,2.055524,35.680523
3,1985.392579,135.885434,6.0,4088.0,1940.0,1978.0,2010.0,2.167978,40.124159
4,1985.219796,133.984398,6.0,4088.0,1941.0,1978.0,2008.0,2.276933,42.497851
...,...,...,...,...,...,...,...,...,...
342,1981.144971,107.048111,157.0,3907.0,1955.0,1979.0,2001.0,2.264916,36.806211
343,1981.301862,109.707060,157.0,3907.0,1954.0,1979.0,2002.0,2.343107,35.746480
344,1981.408157,110.802451,157.0,3907.0,1954.0,1979.0,2002.0,2.359111,34.822753
345,1981.245118,113.459860,157.0,3907.0,1953.0,1979.0,2002.0,2.414423,33.903066
